In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Grindavik,IS,2021-11-30 20:31:01,63.8424,-22.4338,32.47,80,90,21.85
1,1,Victoria,HK,2021-11-30 20:27:41,22.2855,114.1577,62.58,41,38,1.01
2,2,Catamarca,AR,2021-11-30 20:31:01,-28.4696,-65.7852,90.77,33,1,10.00
3,3,Leningradskiy,RU,2021-11-30 20:31:02,69.3833,178.4167,-21.73,96,49,10.09
4,4,Ust-Nera,RU,2021-11-30 20:31:02,64.5667,143.2000,-31.95,99,88,3.98


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
9,9,Vaini,TO,2021-11-30 20:26:12,-21.2000,-175.2000,80.76,83,40,11.50
10,10,Avarua,CK,2021-11-30 20:27:24,-21.2078,-159.7750,80.65,69,40,6.91
14,14,Merauke,ID,2021-11-30 20:31:04,-8.4667,140.3333,79.77,87,100,5.59
19,19,Kimbe,PG,2021-11-30 20:27:22,-5.5502,150.1429,76.33,91,96,3.15
21,21,Vila Velha,BR,2021-11-30 20:28:33,-20.3297,-40.2925,80.55,89,20,8.99
23,23,Nisia Floresta,BR,2021-11-30 20:28:38,-6.0911,-35.2086,85.01,70,77,10.18
24,24,Bathsheba,BB,2021-11-30 20:30:17,13.2167,-59.5167,81.21,83,40,19.57
26,26,Kavieng,PG,2021-11-30 20:27:20,-2.5744,150.7967,82.11,75,98,10.25
31,31,Rikitea,PF,2021-11-30 20:26:13,-23.1203,-134.9692,76.39,75,0,21.05
36,36,Port Hedland,AU,2021-11-30 20:31:07,-20.3167,118.5667,80.65,83,0,13.80


In [11]:
preferred_cities_df.count()

City_ID       168
City          168
Country       167
Date          168
Lat           168
Lng           168
Max Temp      168
Humidity      168
Cloudiness    168
Wind Speed    168
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy().dropna()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Vaini,TO,80.76,-21.2000,-175.2000,
10,Avarua,CK,80.65,-21.2078,-159.7750,
14,Merauke,ID,79.77,-8.4667,140.3333,
19,Kimbe,PG,76.33,-5.5502,150.1429,
21,Vila Velha,BR,80.55,-20.3297,-40.2925,
23,Nisia Floresta,BR,85.01,-6.0911,-35.2086,
24,Bathsheba,BB,81.21,13.2167,-59.5167,
26,Kavieng,PG,82.11,-2.5744,150.7967,
31,Rikitea,PF,76.39,-23.1203,-134.9692,
36,Port Hedland,AU,80.65,-20.3167,118.5667,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]